In [35]:
# Імротуємо біблиотеки
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing

# Завантажуємо датасет
housing = fetch_california_housing()
print(housing['DESCR'])

df = pd.DataFrame(data = np.c_[housing['data'], housing['target']],
                 columns= housing.feature_names + ['target'])

from plotly.graph_objects import *
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.express as px

df.corr()
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
#scatter_matrix(df, figsize =(12,8))
#plt.show()

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split

train,test=train_test_split(df,shuffle=True,test_size=0.2,random_state=33) #ділення дтасету

train_copy=train.copy()
test_copy=test.copy()

x=df["MedInc"].copy()

x=x.values
x=np.array(x).reshape(-1,1)

y=df["target"].copy()
y=y.values

train_x=train_copy["MedInc"].copy()
train_x=train_x.values
train_x=np.array(train_x).reshape(-1,1)
train_y=train_copy["target"].copy()
train_y=train_y.values

test_x=test_copy["MedInc"].copy()
test_x=test_x.values
test_x=np.array(test_x).reshape(-1,1)
test_y=test_copy["target"].copy()
test_y=test_y.values

lin=LinearRegression()
lin.fit(train_x,train_y)
pred_train=lin.predict(train_x)
pred_test=lin.predict(test_x)

#Варіант 1
#plt.plot(x,y, 'gs', train_x,pred_train, 'bs',test_x,pred_test, 'rs')
#plt.show()

#Варіант 2 Закоментувала, бо підвисає
#trace1 = Scatter(x = x.flatten(), y = y, mode='markers', name = 'Raw data')
#trace2 = Scatter(x = train_x.flatten(), y = pred_train, mode = 'markers', name = 'Train data')
#trace3 = Scatter(x = test_x.flatten(), y = pred_test , mode = 'markers', name = 'Test data')
#iplot(Figure(data=[trace1,trace2,trace3]))




from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(train_y, pred_train))

print('mean_absolute_error linear',mean_absolute_error(test_y, pred_test))

print(np.mean(y))
#trace0 = Scatter(x=x.reshape(-1),
#                           y = y,
#                           mode='markers',
#                marker={'color':'blue'},
#                name = 'Raw data')
#fig = Figure(data = [trace0,Scatter()])

#alphas = np.linspace(1,100,100)

#frames = [Frame(data= [Scatter(x = x.reshape(-1),
#                                        y = Ridge(alpha).fit(x,y).predict(x),
#                                        mode = 'lines',
#                                        name = f'alpha = {str(alpha)}'),
#                      trace0]
#                                )for alpha in alphas]
#fig.update(frames=frames)
#fig.update_layout(updatemenus=[dict(type='buttons', showactive=False, buttons=[dict(label='Play', method='animate', args=[None, dict(frame=dict(duration=10, redraw=True), fromcurrent=True, transition=dict(duration=0))])])])

#fig.show()




.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

0.6264890521465973
mean_absolute_error linear 0.6271117739314171
2.068558169089147


In [36]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

mod = RandomForestRegressor()
mod.fit(train_x, train_y)
preds = mod.predict(test_x)
#print(preds)

from sklearn.metrics import r2_score, mean_absolute_error
print(r2_score(test_y, preds))
print('mean_absolute_error tree',mean_absolute_error(test_y,preds))

0.2992604460722632
mean_absolute_error tree 0.7306095683754279


In [ ]:
#за лінейною лігресією помилка менша

In [37]:
import pickle
pickle.dump(mod, open('model_tree.pickle','wb'))
mod = pickle.load(open('model_tree.pickle','rb'))
my_model = {'features':['engine-size'],
           'model': mod}
pickle.dump(my_model, open('my_model_tree.pickle','wb'))
my_model = pickle.load(open('my_model_tree.pickle','rb'))
my_model

{'features': ['engine-size'], 'model': RandomForestRegressor()}

In [38]:
#Варіант 2 https://www.youtube.com/watch?v=Wqmtf9SA_kk
forest = RandomForestRegressor()
forest.fit(train_x, train_y)


RandomForestRegressor()

In [39]:
forest.score(test_x, test_y)


0.30134095215688583

In [40]:
from sklearn.model_selection import GridSearchCV
forest = RandomForestRegressor()
param_grid={"n_estimators":[3,10,30], "max_features":[2,4,6,8]}
grid_search=GridSearchCV(forest,param_grid, cv=5, scoring="neg_mean_squared_error", return_train_score=True)
grid_search.fit(test_x, test_y)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_features': [2, 4, 6, 8],
                         'n_estimators': [3, 10, 30]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [41]:
best_forest=grid_search.best_estimator_
best_forest.score(test_x, test_y)


0.841865162896321

In [42]:
preds_test_best_forest = best_forest.predict(test_x)
preds_train_best_forest = best_forest.predict(train_x)
#print(preds)
print(r2_score(test_y, preds_test_best_forest))
print('mean_absolute_error tree',mean_absolute_error(test_y,preds_test_best_forest))

#в два раза менша помилка ніж за лінейною регресією

0.841865162896321
mean_absolute_error tree 0.3256268201683202


In [43]:


trace1 = Scatter(x = x.flatten(), y = y, mode='markers', name = 'Raw data')
trace2 = Scatter(x = train_x.flatten(), y = preds_train_best_forest, mode = 'markers', name = 'Train data')
trace3 = Scatter(x = test_x.flatten(), y = preds_test_best_forest , mode = 'markers', name = 'Test data')
iplot(Figure(data=[trace1,trace2,trace3]))